In [ ]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
from toniutils import dirtoni as dirt
from toniutils import printoni

<h2>Load Data</h2>

In [1]:
from script import load

In [2]:
path = 'engineered_data/experiment-1/'
X_train, X_val, Y_train, Y_val = [load(path + i + '.pkl') for i in ('X_train', 'X_val', 'Y_train', 'Y_val')]

<h2>Train Model</h2>

In [3]:
import tensorflow as tf

In [100]:
from tensorflow import float32 as fl32
from tensorflow import random_normal as rnorm
from tensorflow import matmul, add
from tensorflow.nn import relu,dropout, tanh
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [31]:
#tf.disable_eager_execution()

In [5]:
batch_size = 32

In [6]:
inputs = tf.placeholder(shape = [batch_size, X_train.shape[1]], dtype = fl32)
labels = tf.placeholder(shape = [batch_size, Y_train.shape[1]], dtype = fl32)

In [127]:
w1 = tf.Variable(rnorm(shape = [inputs.shape[1].value, 110], dtype = fl32, stddev = 0.1))
b1 = tf.Variable(rnorm(shape = [110,], dtype = fl32, stddev = 0.1))
layer1 = dropout( relu(add(matmul(inputs, w1), b1)), rate = 0.1 )

w2 = tf.Variable(rnorm(shape = [layer1.shape[1].value, 220], dtype = fl32, stddev= 0.1))
b2 = tf.Variable(rnorm(shape = [220,], dtype = fl32, stddev = 0.1))
layer2 = dropout( relu(add(matmul(layer1, w2), b2)), rate = 0.1)

w3 = tf.Variable(rnorm(shape = [layer2.shape[1].value, 220], dtype = fl32, stddev= 0.1))
b3 = tf.Variable(rnorm(shape = [220,], dtype = fl32, stddev = 0.1))
layer3 = dropout( relu(add(matmul(layer2, w3), b3)), rate = 0.1)

w4 = tf.Variable(rnorm(shape = [layer2.shape[1].value, 80], dtype = fl32, stddev= 0.1))
b4 = tf.Variable(rnorm(shape = [80,], dtype = fl32, stddev = 0.1))
layer4 = add(matmul(layer3, w4), b4)

In [128]:
loss = tf.reduce_mean(tf.squared_difference(layer4, labels))
loss_minimize = tf.train.AdamOptimizer().minimize(loss)

In [122]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
from toniutils import printoni
from toniutils import dirtoni as dirt

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(200):
    for i in range(X_train.shape[0] // batch_size):
        sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_train[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_train[i * batch_size:(i + 1)*batch_size, :]})

    test_loss = []   
    for i in range(X_val.shape[0] // batch_size):
        new_loss,_ = sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_val[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_val[i * batch_size:(i + 1)*batch_size, :]})
        test_loss.append(new_loss)

    import numpy as np
    mean_test_loss = np.array(test_loss).mean()
    print('Epoch: ' + str(epoch) + '  -  val_loss: ' + str(mean_test_loss))

print(sess.graph)
sess.close()

Epoch: 0  -  val_loss: 0.016237834
Epoch: 1  -  val_loss: 0.013941424
Epoch: 2  -  val_loss: 0.013412267
Epoch: 3  -  val_loss: 0.013136976
Epoch: 4  -  val_loss: 0.01303724
Epoch: 5  -  val_loss: 0.0130083
Epoch: 6  -  val_loss: 0.012979036
Epoch: 7  -  val_loss: 0.012879997
Epoch: 8  -  val_loss: 0.012906579
Epoch: 9  -  val_loss: 0.01278243
Epoch: 10  -  val_loss: 0.012797449
Epoch: 11  -  val_loss: 0.012765304
Epoch: 12  -  val_loss: 0.012699827
Epoch: 13  -  val_loss: 0.012660201
Epoch: 14  -  val_loss: 0.012716565
Epoch: 15  -  val_loss: 0.012711213
Epoch: 16  -  val_loss: 0.012649954
Epoch: 17  -  val_loss: 0.012678682
Epoch: 18  -  val_loss: 0.012641259
Epoch: 19  -  val_loss: 0.012662327
Epoch: 20  -  val_loss: 0.012647987
Epoch: 21  -  val_loss: 0.01257734
Epoch: 22  -  val_loss: 0.012599329
Epoch: 23  -  val_loss: 0.012608099
Epoch: 24  -  val_loss: 0.01258644
Epoch: 25  -  val_loss: 0.0126145985
Epoch: 26  -  val_loss: 0.012616522
Epoch: 27  -  val_loss: 0.012562053
Epoch: 

In [109]:
mean_test_loss

0.013327003

In [115]:
mean_test_loss

0.0135560045

In [124]:
mean_test_loss

0.012763917

<h2>Save Model</h2>

In [119]:
tf.io.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4205122772179942051, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8769741372308327298
 physical_device_desc: "device: XLA_CPU device"]